In [1]:
import pandas as pd 
import numpy as np
import numpy.linalg as lalg
import matplotlib.pyplot as plt 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.regularizers as rg
from sklearn.model_selection import KFold
from Models import *

In [6]:
auto = pd.read_csv("../data/auto-mpg.csv")
auto.fillna(auto.mean())
x = auto[auto.columns[1:7]].to_numpy()
ox = np.insert(x, 0, 1.0, axis = 1)
y = auto[auto.columns[0]].to_numpy()

In [7]:
ridge_perceptron = keras.Sequential()
ridge_perceptron.add(layers.Dense(1, input_dim = 7, 
                                 kernel_initializer = "uniform", 
                                 activation = "relu", 
                                 use_bias = False,
                                 kernel_regularizer = rg.l2(0.01)))

optimizer = keras.optimizers.Adam(learning_rate = 0.0005)
ridge_perceptron.compile(loss = "mean_squared_error", optimizer = optimizer)

In [8]:
ridge_perceptron.fit(ox, y, epochs = 50, batch_size = 10, verbose = 0)
rsq = metrics.rsq(ridge_perceptron, ox, y)
rsq_cv = metrics.rsq_cv(ridge_perceptron, ox, y, epochs = 50)
print(f"Rsq = {rsq} Rsq_cv = {rsq_cv}")

Rsq = 0.5558958947658539 Rsq_cv = 0.7655218794941903


In [24]:
class NeuralNet4L (Perceptron):
    def __init__(self, data, response): 
        """
        Builds a 3 Layer Nueral Network based on the input data. 
        
        Keyword Arguments: 
        
        data -- the x variable matrix 
        response -- the y response vector 
        """
        self.x = data
        self.y = response
        self.k = self.x.shape[1] - 1
        self.n = self.x.shape[0]
        self.model = NeuralNet4L.build_model(self.x, self.y)
    
    def build_model(x, y, suppress = False, width1 = 0, width2 = 0): 
        k = x.shape[1] - 1
        #Default Width Value
        if (width1 == 0): 
            width1 = k + 1
            
        if (width2 == 0): 
            width2 = width1//2
        
        model = keras.Sequential()
        model.add(layers.Dense(width1, input_dim = x.shape[1], 
                               kernel_initializer = "normal", 
                               use_bias = False,
                               activation = "relu", ))
        model.add(layers.Dense(width2,
                              kernel_initializer = "normal", 
                              activation = "relu"))
        model.add(layers.Dense(1, kernel_initializer = "normal"))
        model.compile(loss = "mean_squared_error", optimizer = "adam")
        if not suppress: 
            model.summary()
        return model
    
    def forward_selection(self, iter = 200): 
        return VariableSelection.forward_selection(self.x, self.y, 
                                                   NeuralNet4L.build_model,
                                                   iter)
    
    def backward_elimination(self, iter = 200):
         return VariableSelection.backward_elimination(self.x, self.y, 
                                                   NeuralNet4L.build_model,
                                                   iter)
    
    def stepwise_regression(self, iter = 200): 
         return VariableSelection.stepwise_regression(self.x, self.y, 
                                                      NeuralNet4L.build_model,
                                                      iter)


auto_4L = NeuralNet4L(ox, y)


print(auto_4L.stepwise_regression())

Model: "sequential_169"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_348 (Dense)            (None, 7)                 49        
_________________________________________________________________
dense_349 (Dense)            (None, 3)                 24        
_________________________________________________________________
dense_350 (Dense)            (None, 1)                 4         
Total params: 77
Trainable params: 77
Non-trainable params: 0
_________________________________________________________________
([-6.651878356933594e-05, 0.6908594965934753, 0.707196831703186, 0.7152862250804901, 0.7274307012557983], [-0.0025919392855480172, 0.6892942282217968, 0.7049673659547331, 0.7123883749540829, 0.7239540520371224], [-0.013786303997039794, 0.7124823346734047, 0.7208663627505303, 0.7583192408084869, 0.7565253116190434], [-3299.945213317871, -3765.2073764801025, -3784.8170013427734, -3793.

0.03551584482192993
0.020715576456026064
